In [21]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.1.0


## 모델 저장과 복원

훈련한는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있음. 모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됨. 또 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있음. 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 것들을 제공함.
- 모델을 만드는 코드
- 모델의 훈련된 가중치 또는 파라미터
이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는 데 도움이 됨

주의 : 신뢰할 수 없는 코드는 조심할 것. 텐서플로 모델은 프로그램 코드임. https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md 참고

#### 저장방식 
사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있음. 이 문서는 텐서플로 모델을 만들고 훈련하기 위한 고수준 API인 tf.keras를 사용함. 다른 방법들에 대해서는 텐서플로의 https://www.tensorflow.org/guide/saved_model 문서와 즉시 실행(eager_execution) 문서의 https://www.tensorflow.org/guide/eager#object-based_saving 문서를 참고하자.

### 설정
#### 설치와 임포트
필요한 라이브러리를 설치하고 텐서플로를 임포트함.

pip install -q pyyaml h5py # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다


#### 예제 데이터셋 받기
MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠음.  
모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1000개만 사용함.

In [22]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels= train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) /255.0
test_images = test_images[:1000].reshape(-1,28 * 28) / 255.0

#### 모델 정의
먼저 간단한 모델을 하나 만들어 보자.

In [23]:
# 간단한 Sequential 모델을 정의
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape = (784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(optimizer='adam',
                 loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
                  metrics=['accuracy'])
    
    return model

# 모덱 객체 생성
model = create_model()

# 모델 구조를 출력함
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 훈련하는 동안 체크포인트 저장하기 

훈련 중간과 훈련 마지막에 체크포인트를 자동으로 저장하도록 하는 것이 많이 사용하는 방법임. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있음.
tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백임. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공

#### 체크포인트 콜백 사용하기
훈련하는 동안 가중치를 저장하기 위해 ModelCheckpoint 콜백을 만들어 보자.


In [24]:
import os

In [25]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath= checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
# 새로운 콜백으로 모델 훈련하기
model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data = (test_images, test_labels),
         callbacks=[cp_callback]) # 콜백을 훈련에 전달함
# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있음
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는 ) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 됨


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 544/1000 [===============>..............] - ETA: 0s - loss: 1.5088 - accuracy: 0.5496 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 592us/sample - loss: 1.1551 - accuracy: 0.6660 - val_loss: 0.7038 - val_accuracy: 0.7880
Epoch 2/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.4726 - accuracy: 0.8676
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 269us/sample - loss: 0.4271 - accuracy: 0.8820 - val_loss: 0.5163 - val_accuracy: 0.8380
Epoch 3/10
 512/1000 [==============>...............] - ETA: 0s - loss: 0.2952 - accuracy: 0.9238
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 185us/sample - loss: 0.2843 - accuracy: 0.9290 - val_loss: 0.4530 - val_accuracy: 0.8620
Epoch 4/10
 512/1000 [==============>...............] - ETA: 0s - loss: 0.1910 - accuracy: 0.9609
Epoc

In [26]:
!dir {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FA01-204A

 C:\Users\Kang\GitBig\tensorflow\tensorflow_tutorial\training_1 디렉터리

2020-10-14  오후 11:32    <DIR>          .
2020-10-14  오후 11:32    <DIR>          ..
2020-10-14  오후 11:32                71 checkpoint
2020-10-14  오후 11:32         4,886,712 cp.ckpt.data-00000-of-00001
2020-10-14  오후 11:32             1,222 cp.ckpt.index
               3개 파일           4,888,005 바이트
               2개 디렉터리  97,029,124,096 바이트 남음


훈련하지 않은 새로운 모델을 만들어 보겠음. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야함. 여기서는 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있음

훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가해 보자. 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준 (~10% 정확도)

In [27]:
# 기본 모델 객채 생성
model =create_model()

# 모델을 평가함
loss, acc = model.evaluate(test_images, test_labels,verbose=2)
print('훈련되지 않은 모델의 정확도 : {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 2.3220 - accuracy: 0.1250
훈련되지 않은 모델의 정확도 : 12.50%


체크포인트에서 가중치를 로드하고 다시 평가해보자

In [28]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels,verbose=2)
print('복원된 모델의 정확도 : {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4027 - accuracy: 0.8650
복원된 모델의 정확도 : 86.50%


#### 체크포인트 콜백 매개변수
이 콜백 함수는 몇 가지 매개변수를 제공함. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있음

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보자.

In [29]:
# 파일 이름에 에포크 번호를 포함 시킴('str.format' 포맷)
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듬
cp_callback = tf.keras.callbacks.ModelCheckpoint(
filepath=checkpoint_path,
verbose=1, save_weights_only=True,
period =5)

# 새로운 모델 객체를 만듬
model = create_model()

# 확인
print(checkpoint_path.format(epoch=0))

# 'checkpoint_path' 포맷을 사용하는 가중치를 저장함
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련
model.fit(train_images, train_labels,
         epochs= 50,
         callbacks= [cp_callback],
         validation_data= (test_images, test_labels),
         verbose=0)

training_2/cp-0000.ckpt

Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [30]:
!dir {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FA01-204A

 C:\Users\Kang\GitBig\tensorflow\tensorflow_tutorial\training_2 디렉터리

2020-10-14  오후 11:32    <DIR>          .
2020-10-14  오후 11:32    <DIR>          ..
2020-10-14  오후 11:32                81 checkpoint
2020-10-14  오후 11:32         1,628,745 cp-0000.ckpt.data-00000-of-00001
2020-10-14  오후 11:32               402 cp-0000.ckpt.index
2020-10-14  오후 11:32         4,886,712 cp-0005.ckpt.data-00000-of-00001
2020-10-14  오후 11:32             1,222 cp-0005.ckpt.index
2020-10-14  오후 11:32         4,886,712 cp-0010.ckpt.data-00000-of-00001
2020-10-14  오후 11:32             1,222 cp-0010.ckpt.index
2020-10-14  오후 11:32         4,886,712 cp-0015.ckpt.data-00000-of-00001
2020-10-14  오후 11:32             1,222 cp-0015.ckpt.index
2020-10-14  오후 11:32         4,886,712 cp-0020.ckpt.data-00000-of-00001
2020-10-14  오후 11:32             1,222 cp-0020.ckpt.index
2020-10-14  오후 11:32         4,886,712 cp-0025.ckpt.data-00000-of-00001
2020-10-14  오후 11:32         

In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

노트 : 텐서플로는 기본적으로 최근 5개의 체크포인트만 저장함.
    
모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보자.

In [32]:
# 새로운 모델 객체 생성
model =create_model()

# 이전에 저장한 최신 가중치를 로드
model.load_weights(latest)

# 모델을 평가
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도 {:5.2f}%".format(100 *acc))

1000/1000 - 0s - loss: 0.4956 - accuracy: 0.8740
복원된 모델의 정확도 87.40%


### 이 파일들은 무엇인가요?
위 코드는 이진 형식의 훈련된 가중치만 포함하는 체크포인트 형식의 파일에 가중치를 저장함.  
체크포인트의 내용:  
    - 모델의 중량이 들어 있는 파편 하나 이상.
    - 어떤 가중치가 어느 조각에 있는지 나타내는 인덱스 파일
    
만약 한 대의 모델을 훈련 했다면, 접미사가 있는 하나의 조각만 있을 것임. data-00000-of-00001

### 수동으로 가중치 저장하기
앞에서 가중치를 모델에 로드하는 방법을 봄. 수동으로 가중치를 저장하는 것도 쉬움. Model_save_weights 메서드를 사용함. tf.keras는, 특히 save_weights는 .ckpt 확장자를 가진 텐서플로 체크포인트 포맷을 사용함. (.h5 확장자의 HDF5으로 저장하는 것은 https://www.tensorflow.org/guide/keras/save_and_serialize#weights_only_saving_in_savedmodel_format 가이드에서 다룸):


In [33]:
# 가중치를 저장함
model.save_weights('./checkpoints/my_checkpint')

# 새로운 모델 객체 생성
model = create_model()

# 가중치를 복원함
model.load_weights('./checkpoints/my_checkpint')

# 모델을 평가함
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('복원된 모델의 정확도 : {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4956 - accuracy: 0.8740
복원된 모델의 정확도 : 87.40%


### 전체 모델 저장하기

model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장함. 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있음. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있음.

두 개의 포맷(SaveModel과 HDF5)으로 모델을 저장할 수 있음. 텐서플로의 SaveModel 포맷은 TF2.X에서 기본 파일 포맷임. 하지만 HDF5 포맷으로 저장 가능. 두 파일 포맷으로 전체 모델을 저장하는 방법은 아래에서 자세히 설명함.

전체 모델을 저장하는 기능은 매우 유용함. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있음(Saved Model, HDF5) 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행 가능

* 사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요함. 아래 사용자 정의 객체 저장하기 섹션을 참고

#### SavedModel 포맷

SavedModel 포맷은 모델을 직렬화하는 다른 방법임. 이 포맷으로 저장한 모델은 tf.keras.models.load_mdoel로 복원할 수 있고 텐서플로 서빙과 호환됨. SavedModel 가이드(https://www.tensorflow.org/guide/saved_model)에서 SavedModel를 서빙하고 점검하는 자세한 방법을 제공함 이 섹션에서는 모델을 저장하고 복원하는 방법을 안내함.

In [39]:
# 새로운 모델을 객체를 만들고 훈련함.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장함
!mkdir saved_model

model.save('./saved_model')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 442us/sample - loss: 1.1769 - accuracy: 0.6580
Epoch 2/5
1000/1000 [==============================] - 0s 108us/sample - loss: 0.4378 - accuracy: 0.8740
Epoch 3/5
1000/1000 [==============================] - 0s 118us/sample - loss: 0.2942 - accuracy: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 98us/sample - loss: 0.2085 - accuracy: 0.9450
Epoch 5/5
1000/1000 [==============================] - 0s 98us/sample - loss: 0.1586 - accuracy: 0.9680


하위 디렉터리 또는 파일 saved_model이(가) 이미 있습니다.


INFO:tensorflow:Assets written to: ./saved_model\assets


SavedModel 포맷은 프로토콜 버퍼 이진 파일과 텐서플로 체크포인트를 담고 있는 디렉토리임.  
저장된 모델의 디렐토리를 확인해보자

In [41]:
# my_model 디렉토리 /#assests 폴더, saved_model.pb, variables 폴더
!dir saved_model


 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: FA01-204A

 C:\Users\Kang\GitBig\tensorflow\tensorflow_tutorial\saved_model 디렉터리

2020-10-14  오후 11:42    <DIR>          .
2020-10-14  오후 11:42    <DIR>          ..
2020-10-14  오후 11:42    <DIR>          assets
2020-10-14  오후 11:42            83,004 saved_model.pb
2020-10-14  오후 11:42    <DIR>          variables
               1개 파일              83,004 바이트
               4개 디렉터리  97,014,677,504 바이트 남음


저장된 모델로부터 새로운 케라스 모델을 로드함.

In [42]:
new_model = tf.keras.models.load_model('saved_model')

new_model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있음. 이 모델을 평가하고 예측에 사용해보자.

In [44]:
# 복원된 모델을 평가함
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.4345 - accuracy: 0.8560
복원된 모델의 정확도: 85.60%
(1000, 10)


#### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공함


In [45]:
# 새로운 모델 객체를 만들고 훈련함
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장함
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냄
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 438us/sample - loss: 1.1415 - accuracy: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 108us/sample - loss: 0.4376 - accuracy: 0.8700
Epoch 3/5
1000/1000 [==============================] - 0s 103us/sample - loss: 0.2768 - accuracy: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 100us/sample - loss: 0.1950 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 0s 102us/sample - loss: 0.1560 - accuracy: 0.9660


이제 이 파일로부터 모델을 다시 만들어 보자.

In [46]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성함
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력함
new_model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


정확도를 확인해 보겠음

In [47]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4460 - accuracy: 0.8640
복원된 모델의 정확도: 86.40%


Keras는 모델 구조를 파악하여 저장함. 이 방식은 다음과 같은 모든 것을 저장함.
- 가중치 값
- 모델 구조
- 모델의 훈련 설정(컴파일 시 전달한 값)
- 옵티마이저와 필요한 옵티마이저 상태( 중지된 시점부터 다시 시작할 수 있음)

체크포인트가 호환되지 않기 때문에 케라슨 v1.x옵티마이저
(tf.compact.v1.train)를 저장할 수 없음. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 함. 따라서 옵티마이저의 상태를 잃게 됨

### 사용자 정의 객체
SaveModel 포맷을 사용한다면 이 섹션은 건너 뛰어도 됩니다. HDF5와 SavedModel 사이의 주요 차이점은 HDF5가 객체로 모델 구조를 저장하고 SavedModel은 실행 그래프로 저장하는 것입니다. 따라서 SavedModel은 원본 코드 없이 상속 모델과 사용자 정의 층 같은 사용자 정의 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르세요:

1. 이 객체에 get_config 메서드를 정의하고 선택적으로 from_config 클래스 메서드를 정의합니다.
    - get_config(self)는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
    - from_config(cls, config)는 get_config에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(return cls(**config)).
2. 모델을 로드할 때 이 객체를 custom_objects 매개변수로 전달합니다. 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다. 예를 들면 tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})

사용자 정의 객체와 get_config에 관한 예제를 보려면 Writing layers and models from scratch 튜토리얼을 참고하세요.

In [ ]:
# 저작권
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.